### Pre-installs

In [1]:
# %pip install pandas
# %pip install numpy
# %pip install matplotlib
# %pip install seaborn
# %pip install sklearn
# %pip install -U scikit-learn
# %pip install spotipy

### Inputs

In [2]:
input_song = input("Enter your desired song name here!")
input_playlist = input("Enter your desired playlist name here!s")
recc_num = int(input("Enter how many songs you want to be recommended!"))
client_id = input("Enter your Spotify API Client Key here!")
client_secret = input("Enter your Spotify API Secret Key here!")
sp_dc = input("Enter your Spotify sp_dc here!")
sp_key = input("Enter your Spotify sp_key here!")

### CSV & Spotify Information

In [3]:
import pandas as pd
import ast

# Importing the datasets
dataset = pd.read_csv("data/tracks_with_genres_&_language.csv")
df = pd.read_csv("data/tracks.csv")

# Creating the dataframe
df_generated = pd.DataFrame(dataset)
df_generated['genres'] = df_generated['genres'].apply(ast.literal_eval)

In [4]:
df_generated.iloc[:,3:19].head()

,duration_ms,explicit,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,160187,0,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,0,-11.101,1,0.0322,0.394,0.000000,0.149,0.285,113.564,3
1,159560,0,['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,0.867,2,-2.706,1,0.0571,0.436,0.000000,0.139,0.839,120.689,4
2,141987,0,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,6,-10.226,0,0.0289,0.255,0.000005,0.163,0.588,104.536,4
3,157093,0,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.477,0.352,1,-14.165,1,0.0300,0.406,0.000000,0.122,0.478,106.773,4
4,187333,0,['1Mxqyy3pSjf8kZZL4QVxS0'],2018-05-04,0.319,0.201,7,-17.796,1,0.0623,0.887,0.000000,0.904,0.239,117.153,3


In [5]:
# Spotify API Authentication Information
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth
# api key
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri="http://localhost/",
                                               scope="playlist-modify-private",
                                               show_dialog=True,
                                               cache_path="token.txt"
                                               ))

In [6]:
import seaborn as sns
import matplotlib.pyplot as plt

# # Assuming df_generated is your DataFrame
# # Adjust this part based on your actual DataFrame structure
# # For example, you might need to select specific columns or rows
# # or convert categorical variables to numerical values for visualization

# df_generated.columns
# df_graph = df_generated[['popularity', 'duration_ms', 'explicit',
#        'danceability', 'energy', 'key', 'loudness',
#        'speechiness', 'acousticness', 'instrumentalness', 'liveness',
#        'valence', 'tempo', 'time_signature']]

# sns.set(style="white")  # Set the style of the plot

# # Create a heatmap using seaborn with annotated values for the entire correlation matrix
# plt.figure(figsize=(12, 10))  # Adjust the figure size as needed
# heatmap = sns.heatmap(df_graph.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)

# # Customize the appearance of the heatmap
# heatmap.set_title('Heatmap of df_generated', fontsize=16)

# plt.show()

### Processing the Input Song

In [7]:
# Using Spotify API to search for a song's information based on input and adding the necessary information in a DataFrame.
def search_track(track_name):
    # Search for the track
    results = sp.search(q=track_name, type='track')

    # Check if the track exists
    if results['tracks']['total'] > 0:
        # Get the first track from the results
        track = results['tracks']['items'][0]
        explicit = int(track["explicit"] == True)
        artists = []
        id_artists = []
        for i in range (len(track["artists"])):
            artists.append(track["artists"][i]["name"])
            id_artists.append(track["artists"][i]["id"])

        print(f'Found track: {track["name"]} by {track["artists"][0]["name"]} from the album {track["album"]["name"]}.')
        track_dict = {"id": track["id"], "name": track["name"], "popularity": track["popularity"], 
                      "duration_ms": track["duration_ms"], "explicit": explicit, "artists": str(artists),
                      "id_artists": str(id_artists), "release_date": track["album"]["release_date"]}
        return track_dict
    else:
        print('Track not found')
        return None

# Creating a Single Row DataFrame for the input song.
track_result = search_track(input_song)
td = pd.DataFrame(track_result, index=[0])
td

Found track: God knows... by 涼宮ハルヒ (CV.平野 綾) from the album 涼宮ハルヒの詰合.


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date
0,4YnXlBbFnMJf6qiO2LQMlU,God knows...,56,279040,0,['涼宮ハルヒ (CV.平野 綾)'],['4eWMhmDkvSkgeHo0D2RT31'],2006-06-21


In [8]:
# Obtaining Feature Data from song based on its song_id from previous function and adding them to a DataFrame.
def get_audio_features(track_result):
    song_id = track_result["id"]
    results = sp.audio_features(song_id)

    if results:
        return results[0]
    else:
        print(f'No audio features found for song ID: {song_id}')
        return None

audio_features = get_audio_features(track_result)
af = pd.DataFrame(audio_features, index=[0])
# Taking out the irrevelant features.
af_formatted = af.drop(["type", "id", "uri", "track_href", "analysis_url", "duration_ms"], axis=1)
# Merging both DataFrames to sync up with the dataset's layout.
td = pd.concat([td, af_formatted], axis=1)
td['year'] = td['release_date'].str.extract(r'(\d{4})').astype(int)

In [9]:
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year
0,4YnXlBbFnMJf6qiO2LQMlU,God knows...,56,279040,0,['涼宮ハルヒ (CV.平野 綾)'],['4eWMhmDkvSkgeHo0D2RT31'],2006-06-21,0.456,0.955,...,-2.225,1,0.133,0.00152,0.000007,0.351,0.411,150.01,4,2006


In [10]:
import ast

# Converting string array into regular array.
td['artists'] = td['artists'].apply(ast.literal_eval)

td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year
0,4YnXlBbFnMJf6qiO2LQMlU,God knows...,56,279040,0,[涼宮ハルヒ (CV.平野 綾)],['4eWMhmDkvSkgeHo0D2RT31'],2006-06-21,0.456,0.955,...,-2.225,1,0.133,0.00152,0.000007,0.351,0.411,150.01,4,2006


In [11]:
# Viewing the input song features.
td['artists_count'] = len(td['artists'][0])
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count
0,4YnXlBbFnMJf6qiO2LQMlU,God knows...,56,279040,0,[涼宮ハルヒ (CV.平野 綾)],['4eWMhmDkvSkgeHo0D2RT31'],2006-06-21,0.456,0.955,...,1,0.133,0.00152,0.000007,0.351,0.411,150.01,4,2006,1


In [12]:
# Extracting the artists from the input artists array.
expanded_artists = td['artists'].apply(lambda x: pd.Series(x))

# Rename the columns
expanded_artists.columns = [f"artist_{i+1}" for i in range(len(expanded_artists.columns))]

# Concatenate the expanded columns with the original DataFrame
td = pd.concat([td, expanded_artists], axis=1)
td.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1
0,4YnXlBbFnMJf6qiO2LQMlU,God knows...,56,279040,0,[涼宮ハルヒ (CV.平野 綾)],['4eWMhmDkvSkgeHo0D2RT31'],2006-06-21,0.456,0.955,...,0.133,0.00152,0.000007,0.351,0.411,150.01,4,2006,1,涼宮ハルヒ (CV.平野 綾)


In [13]:
# Obtaining input artist(s) genre(s) through the Spotify API.
def get_artist_genres(artist_name):
    try:
        results = sp.search(q='artist:' + artist_name, type='artist')
        genres = results['artists']['items'][0]['genres']
        return genres if genres else []
    except IndexError:
        return []
    
for i in range(td['artists_count'][0]):
    td['genres' + str(i+1)] = td['artist_' + str(i+1)].apply(get_artist_genres)
    td['genres' + str(i+1)] = td['genres' + str(i+1)].fillna('[]')
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1,genres1
0,4YnXlBbFnMJf6qiO2LQMlU,God knows...,56,279040,0,[涼宮ハルヒ (CV.平野 綾)],['4eWMhmDkvSkgeHo0D2RT31'],2006-06-21,0.456,0.955,...,0.00152,0.000007,0.351,0.411,150.01,4,2006,1,涼宮ハルヒ (CV.平野 綾),[shonen]


In [14]:
# Combining all genres into one list.
for i in range(td['artists_count'][0]):
    td['genres'] = td['genres' + str(i+1)] + td['genres' + str(i+1)]
    td.drop(['genres' + str(i+1)], axis=1, inplace=True)
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1,genres
0,4YnXlBbFnMJf6qiO2LQMlU,God knows...,56,279040,0,[涼宮ハルヒ (CV.平野 綾)],['4eWMhmDkvSkgeHo0D2RT31'],2006-06-21,0.456,0.955,...,0.00152,0.000007,0.351,0.411,150.01,4,2006,1,涼宮ハルヒ (CV.平野 綾),"[shonen, shonen]"


In [15]:
# Returning only the unique genres for the input.
def get_unique(row):
    return list(set(row))

td['genres'] = td['genres'].apply(get_unique)

In [16]:
# Determining the language from a Lyrics Scraping Library.
import spotify_lyrics_scraper as spotify

token = spotify.getToken(sp_dc, sp_key)
lyrics_data = spotify.getLyrics(token, songName=td['name'][0])
if lyrics_data['status'] == False:
        language = "null"
else:
    # Extract the language from the lyrics data
    language = lyrics_data['message']['lyrics']['language']

td['language'] = language
td

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1,genres,language
0,4YnXlBbFnMJf6qiO2LQMlU,God knows...,56,279040,0,[涼宮ハルヒ (CV.平野 綾)],['4eWMhmDkvSkgeHo0D2RT31'],2006-06-21,0.456,0.955,...,0.000007,0.351,0.411,150.01,4,2006,1,涼宮ハルヒ (CV.平野 綾),[shonen],ja


In [17]:
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

# Many songs do not have lyrics enabled by spotify. So the alternative to that is to use their name as a benchmark for the language.
# Uses language detect library to identify the song name's expected language. Not exactly accurate but it is an acceptable alternative for now until NLP.
def replace_nan_with_language(row):
    if pd.isna(row['language']):  # Check if the value is NaN
        name = row['name']
        if isinstance(name, str) and len(name) > 0:  # Check if name is a non-empty string
            try:
                return detect(name)  # Use detect() output as the value if it's NaN
            except LangDetectException:
                pass  # Handle LangDetectException, e.g., return 'unknown' or None
        # Return None or any other default value if language detection fails or input text is empty
        return None  
    else:
        return row['language']  # Otherwise, keep the original value

td['language'] = td.apply(replace_nan_with_language, axis=1)

td.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,instrumentalness,liveness,valence,tempo,time_signature,year,artists_count,artist_1,genres,language
0,4YnXlBbFnMJf6qiO2LQMlU,God knows...,56,279040,0,[涼宮ハルヒ (CV.平野 綾)],['4eWMhmDkvSkgeHo0D2RT31'],2006-06-21,0.456,0.955,...,0.000007,0.351,0.411,150.01,4,2006,1,涼宮ハルヒ (CV.平野 綾),[shonen],ja


In [18]:
# Creating a unique identifier.
td['song_name_artist'] = td['name'] + str(td['artists'])

In [19]:
# Defining the values to give priority to later.
language_prority = td['language'][0]
key_priority = td['key'][0]
genre_priority = td['genres'][0]

### Adding Input to Dataset and further processing

In [20]:
# Adding the Input Song to the Dataset DataFrame. Added to the very front.
new_df = pd.concat([td, df_generated], ignore_index=True)
new_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,time_signature,year,artists_count,artist_1,genres,language,song_name_artist,artist_2,artist_3,artist_4
0,4YnXlBbFnMJf6qiO2LQMlU,God knows...,56,279040,0,[涼宮ハルヒ (CV.平野 綾)],['4eWMhmDkvSkgeHo0D2RT31'],2006-06-21,0.456,0.955,...,4,2006,1,涼宮ハルヒ (CV.平野 綾),[shonen],ja,God knows...0 [涼宮ハルヒ (CV.平野 綾)]\nName: arti...,NaN,NaN,NaN
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,56,160187,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,...,3,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,You'll Never Walk Alone - Mono; 2002 Remaster[...,NaN,NaN,NaN
2,6Pkt6qVikqPBt9bEQy8iTz,A Lover's Concerto,41,159560,0,NaN,['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,0.867,...,4,2020,1,The Toys,[],en,A Lover's Concerto['The Toys'],NaN,NaN,NaN
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,40,141987,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,...,4,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Ferry Cross the Mersey - Mono; 2002 Remaster['...,NaN,NaN,NaN
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,34,157093,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.477,0.352,...,4,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Don't Let the Sun Catch You Crying (Main) - Mo...,NaN,NaN,NaN


In [21]:
new_df['langauge'] = new_df['language'].fillna("null")
new_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,year,artists_count,artist_1,genres,language,song_name_artist,artist_2,artist_3,artist_4,langauge
0,4YnXlBbFnMJf6qiO2LQMlU,God knows...,56,279040,0,[涼宮ハルヒ (CV.平野 綾)],['4eWMhmDkvSkgeHo0D2RT31'],2006-06-21,0.456,0.955,...,2006,1,涼宮ハルヒ (CV.平野 綾),[shonen],ja,God knows...0 [涼宮ハルヒ (CV.平野 綾)]\nName: arti...,NaN,NaN,NaN,ja
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,56,160187,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,...,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,You'll Never Walk Alone - Mono; 2002 Remaster[...,NaN,NaN,NaN,en
2,6Pkt6qVikqPBt9bEQy8iTz,A Lover's Concerto,41,159560,0,NaN,['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,0.867,...,2020,1,The Toys,[],en,A Lover's Concerto['The Toys'],NaN,NaN,NaN,en
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,40,141987,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,...,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Ferry Cross the Mersey - Mono; 2002 Remaster['...,NaN,NaN,NaN,en
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,34,157093,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.477,0.352,...,2008,1,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Don't Let the Sun Catch You Crying (Main) - Mo...,NaN,NaN,NaN,en


In [22]:
new_df['song_id_name'] = new_df['id'] + new_df['name']
print(new_df.shape)
new_df.drop_duplicates(subset=["song_id_name"], keep='first', inplace=True)
print(new_df.shape)

(186012, 31)
(186011, 31)


In [23]:
# Counting the number of genres in for each song.
def count_items_in_list(lst):
    return len(lst)

# Apply the function to the DataFrame column
new_df['genres_count'] = new_df['genres'].apply(count_items_in_list)
new_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,artist_1,genres,language,song_name_artist,artist_2,artist_3,artist_4,langauge,song_id_name,genres_count
0,4YnXlBbFnMJf6qiO2LQMlU,God knows...,56,279040,0,[涼宮ハルヒ (CV.平野 綾)],['4eWMhmDkvSkgeHo0D2RT31'],2006-06-21,0.456,0.955,...,涼宮ハルヒ (CV.平野 綾),[shonen],ja,God knows...0 [涼宮ハルヒ (CV.平野 綾)]\nName: arti...,NaN,NaN,NaN,ja,4YnXlBbFnMJf6qiO2LQMlUGod knows...,1
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,56,160187,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,...,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,You'll Never Walk Alone - Mono; 2002 Remaster[...,NaN,NaN,NaN,en,6catF1lDhNTjjGa2GxRQNNYou'll Never Walk Alone ...,9
2,6Pkt6qVikqPBt9bEQy8iTz,A Lover's Concerto,41,159560,0,NaN,['6lH5PpuiMa5SpfjoIOlwCS'],2020-03-13,0.671,0.867,...,The Toys,[],en,A Lover's Concerto['The Toys'],NaN,NaN,NaN,en,6Pkt6qVikqPBt9bEQy8iTzA Lover's Concerto,0
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,40,141987,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,...,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Ferry Cross the Mersey - Mono; 2002 Remaster['...,NaN,NaN,NaN,en,4aSw1QJIMwYSoDEgzgdCJLFerry Cross the Mersey -...,9
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,34,157093,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.477,0.352,...,Gerry & The Pacemakers,"[adult standards, bubblegum pop, merseybeat, r...",en,Don't Let the Sun Catch You Crying (Main) - Mo...,NaN,NaN,NaN,en,0ZMMtH875IR2TfkyC4PolDDon't Let the Sun Catch ...,9


In [24]:
print(new_df.shape)
new_df = new_df[(new_df['genres_count'] != 0)]
print(new_df.shape)

(186011, 32)
(166251, 32)


### Supersetting genres

In [25]:
# Determining the frequency of the genre. Giving the input song's genre priority to go first.
genre_extract = new_df['genres'].reset_index()
genre_wo_index = genre_extract['genres']
tally = genre_wo_index.count()
genre_dict = {}

for i in range (tally):
    input = genre_wo_index.loc[i]
    if len(input) > 0:
        for n in input:
            if n in genre_dict.keys():
                genre_dict[n] += 1
            else:
                genre_dict[n] = 1

print(genre_dict)

{'shonen': 13, 'adult standards': 1216, 'bubblegum pop': 416, 'merseybeat': 529, 'rock-and-roll': 650, 'british invasion': 495, 'classic uk pop': 704, 'brill building pop': 763, 'rockabilly': 550, 'folk rock': 933, 'easy listening': 126, 'lounge': 283, 'british blues': 265, 'canadian blues': 54, 'singer-songwriter': 312, 'lilith': 307, 'canadian singer-songwriter': 107, 'folk': 393, 'rock': 3193, 'mellow gold': 1141, 'jazz': 304, 'swing': 56, 'big band': 94, 'jazz piano': 73, 'stride': 75, 'vocal jazz': 355, 'cool jazz': 274, 'harlem renaissance': 23, 'jazz saxophone': 71, 'contemporary jazz': 73, 'jazz quartet': 20, 'jazz fusion': 297, 'swedish jazz': 134, 'free jazz': 58, 'avant-garde jazz': 49, 'contemporary post-bop': 57, 'italian jazz': 15, 'soundtrack': 456, 'italian soundtrack': 28, 'vintage italian soundtrack': 34, 'classic soundtrack': 89, 'jazz trumpet': 176, 'hard bop': 204, 'roots rock': 250, 'psychedelic rock': 592, 'classic rock': 1358, 'acid rock': 67, 'electric blues': 

In [26]:
genres_df = pd.Series(genre_dict)
genres_df.head()

shonen               13
adult standards    1216
bubblegum pop       416
merseybeat          529
rock-and-roll       650
dtype: int64

In [27]:
# Summary Statistics of genres.
genres_df.T.describe()

count    4229.000000
mean      130.132183
std       393.310202
min         1.000000
25%         4.000000
50%        19.000000
75%        85.000000
max      6880.000000
dtype: float64

In [28]:
# genres_df.to_csv("data/genre_frequency.csv")

In [29]:
# Creating the genre supersets, with priority to the input song's genre.
# Since the input song is the first one to be run, its genre will be able to start the classification of other sub-genres under it.

# This can be further improved. Allowing for later genres to take over earlier genres as the superset if it more general.
# E.g.: Detroit Hip Hop is found before general hip hop. Currently they will be seperated into 2 seperate genres because the first word 'detroit' is
# not 'hip hop' and is thus overlooked.

def generate_superset_mapping(genre_dict):
    superset_mapping = {}

    for genre, frequency in genre_dict.items():
        subset_added = False
        for superset in list(superset_mapping.keys()):
            if genre in superset:
                superset_mapping[genre] = [genre]
                superset_mapping[genre].extend(superset_mapping[superset])
                del superset_mapping[superset]
                subset_added = True
                break
            elif superset in genre:
                if superset in superset_mapping:
                    superset_mapping[superset].append(genre)
                    subset_added = True
                    break
        if not subset_added:
            superset_mapping[genre] = [genre]

    return superset_mapping

# Generate superset mapping
superset_mapping = generate_superset_mapping(genre_dict)

# Output the superset mapping
for superset, subsets in superset_mapping.items():
    print(f"{superset}: {subsets}")


shonen: ['shonen']
adult standards: ['adult standards', 'deep adult standards']
merseybeat: ['merseybeat']
british invasion: ['british invasion']
brill building pop: ['brill building pop']
rockabilly: ['rockabilly', 'uk rockabilly', 'finnish rockabilly', 'german rockabilly', 'neo-rockabilly', 'swedish rockabilly', 'spanish rockabilly', 'rockabilly en espanol', 'japanese rockabilly', 'canadian rockabilly']
easy listening: ['easy listening']
lounge: ['lounge', 'lounge house', 'chill lounge', 'italian lounge', 'sunset lounge']
canadian blues: ['canadian blues']
singer-songwriter: ['singer-songwriter', 'canadian singer-songwriter', 'nashville singer-songwriter', 'taiwan singer-songwriter', 'singaporean singer-songwriter', 'indian singer-songwriter', 'scottish singer-songwriter', 'japanese singer-songwriter', 'swedish singer-songwriter', 'british singer-songwriter', 'turkish singer-songwriter', 'irish singer-songwriter', 'norwegian singer-songwriter', 'nz singer-songwriter', 'korean singer-

In [30]:
# Supersetting the weighted genres for later.

superset_genre_priority = []
if len(genre_priority) != 0:
    for i in genre_priority:
        for n in superset_mapping:
            if i in n:
                superset_genre_priority.append(n)
print(superset_genre_priority)

['shonen']


In [31]:
# Now with the superset mapped out, apply it onto the all the songs to determine their supersetted genre(s).
# This can be improved. Instead of only returning the first superset genre, it should return multiple in an array.

import pandas as pd

new_df['supersetted_genres'] = new_df['genres'].apply(lambda genres: [m for n in genres for m in superset_mapping if n in m])

In [32]:
new_df['supersetted_genres'].value_counts().count()

6104

In [33]:
new_df.head()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,genres,language,song_name_artist,artist_2,artist_3,artist_4,langauge,song_id_name,genres_count,supersetted_genres
0,4YnXlBbFnMJf6qiO2LQMlU,God knows...,56,279040,0,[涼宮ハルヒ (CV.平野 綾)],['4eWMhmDkvSkgeHo0D2RT31'],2006-06-21,0.456,0.955,...,[shonen],ja,God knows...0 [涼宮ハルヒ (CV.平野 綾)]\nName: arti...,NaN,NaN,NaN,ja,4YnXlBbFnMJf6qiO2LQMlUGod knows...,1,[shonen]
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,56,160187,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.484,0.265,...,"[adult standards, bubblegum pop, merseybeat, r...",en,You'll Never Walk Alone - Mono; 2002 Remaster[...,NaN,NaN,NaN,en,6catF1lDhNTjjGa2GxRQNNYou'll Never Walk Alone ...,9,"[adult standards, merseybeat, british invasion..."
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,40,141987,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.405,0.365,...,"[adult standards, bubblegum pop, merseybeat, r...",en,Ferry Cross the Mersey - Mono; 2002 Remaster['...,NaN,NaN,NaN,en,4aSw1QJIMwYSoDEgzgdCJLFerry Cross the Mersey -...,9,"[adult standards, merseybeat, british invasion..."
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,34,157093,0,NaN,['3UmBeGyNwr4iDWi1vTxWi8'],2008-02-11,0.477,0.352,...,"[adult standards, bubblegum pop, merseybeat, r...",en,Don't Let the Sun Catch You Crying (Main) - Mo...,NaN,NaN,NaN,en,0ZMMtH875IR2TfkyC4PolDDon't Let the Sun Catch ...,9,"[adult standards, merseybeat, british invasion..."
5,1hx7X9cMXHWJjknb9O6Ava,The September Of My Years - Live At The Sands ...,26,187333,0,NaN,['1Mxqyy3pSjf8kZZL4QVxS0'],2018-05-04,0.319,0.201,...,"[easy listening, adult standards, lounge]",en,The September Of My Years - Live At The Sands ...,NaN,NaN,NaN,en,1hx7X9cMXHWJjknb9O6AvaThe September Of My Year...,3,"[easy listening, adult standards, lounge]"


### Encoding

In [34]:
# # One-hot Encoding of all the superset genres of each song in the dataframe.

# genres_dummies = pd.get_dummies(new_df['supersetted_genres'].apply(pd.Series).stack(), prefix="genre").groupby(level=0).sum()

# # Concatenate the binary columns with the original DataFrame
# df_2 = pd.concat([new_df, genres_dummies], axis=1)

In [35]:
# df_2.iloc[:,0:20].describe()

In [36]:
# Determining how many keys there are.
new_df.groupby('key').count()

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,...,genres,language,song_name_artist,artist_2,artist_3,artist_4,langauge,song_id_name,genres_count,supersetted_genres
key,,,,,,,,,,,,,,,,,,,,,
0,19586,19586,19586,19586,19586,0,19586,19586,19586,19586,...,19586,19535,19586,4082,1013,194,19586,19586,19586,19586
1,14502,14502,14502,14502,14502,0,14502,14502,14502,14502,...,14502,14446,14502,3955,1026,220,14502,14502,14502,14502
2,17337,17337,17337,17337,17337,0,17337,17337,17337,17337,...,17337,17297,17337,3440,921,164,17337,17337,17337,17337
3,5024,5024,5024,5024,5024,0,5024,5024,5024,5024,...,5024,5004,5024,1090,271,37,5024,5024,5024,5024
4,13626,13626,13626,13626,13626,0,13626,13626,13626,13626,...,13626,13586,13626,2714,646,126,13626,13626,13626,13626
5,13199,13199,13199,13199,13199,0,13199,13199,13199,13199,...,13199,13165,13199,2882,709,162,13199,13199,13199,13199
6,11252,11252,11252,11252,11252,0,11252,11252,11252,11252,...,11252,11208,11252,2821,708,152,11252,11252,11252,11252
7,19669,19669,19669,19669,19669,0,19669,19669,19669,19669,...,19669,19618,19669,4162,1053,215,19669,19669,19669,19669
8,10079,10079,10079,10079,10079,0,10079,10079,10079,10079,...,10079,10042,10079,2474,646,140,10079,10079,10079,10079


In [37]:
new_df['key'].max()

11

In [38]:
# One-hot encoding the keys.
key_encoded = pd.get_dummies(new_df['key'], columns=[i for i in range(12)], prefix='key').astype(int)

In [39]:
key_encoded.head()

,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,1,0,0,0,0


In [40]:
# Adding keys encoding into the dataset.
df_2 = pd.concat([new_df, key_encoded], axis=1)

In [41]:
print(df_2.shape)
df_2.drop_duplicates(subset=["song_id_name"], keep='first', inplace=True)
print(df_2.shape)

(166251, 45)
(166251, 45)


In [42]:
# # Checking all the languages.
# df_2['langauge'].value_counts().keys()

In [43]:
# df_2['language'].head()

In [44]:
# # Encoding all the languages.
# df_2['language'] = df_2['language'].fillna('null')
# language_encoded = pd.get_dummies(df_2['language'], prefix='lang').astype(int)
# language_encoded.head()

In [45]:
# language_encoded = language_encoded.drop(['lang_null'], axis=1)
# language_encoded.head()

In [46]:
# # Adding language encoding into the dataset.
# df_2 = pd.concat([df_2, language_encoded], axis=1)

In [47]:
# print(df_2.shape)
# df_2.drop_duplicates(subset=["song_id_name"], keep='first', inplace=True)
# print(df_2.shape)

In [48]:
# Removing unnecessary features.
df_2 = df_2.drop(['id_artists', 'release_date', 'popularity', 'key', 'artists_count', 'song_name_artist', 'artist_1', 'artist_2', 'artist_3', 'artist_4', 'genres', 'explicit', 'time_signature'], axis=1)
df_2.head()

,id,name,duration_ms,artists,danceability,energy,loudness,mode,speechiness,acousticness,...,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,4YnXlBbFnMJf6qiO2LQMlU,God knows...,279040,[涼宮ハルヒ (CV.平野 綾)],0.456,0.955,-2.225,1,0.1330,0.00152,...,0,0,0,0,0,0,0,0,0,1
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,160187,NaN,0.484,0.265,-11.101,1,0.0322,0.39400,...,0,0,0,0,0,0,0,0,0,0
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,141987,NaN,0.405,0.365,-10.226,0,0.0289,0.25500,...,0,0,0,0,1,0,0,0,0,0
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,157093,NaN,0.477,0.352,-14.165,1,0.0300,0.40600,...,0,0,0,0,0,0,0,0,0,0
5,1hx7X9cMXHWJjknb9O6Ava,The September Of My Years - Live At The Sands ...,187333,NaN,0.319,0.201,-17.796,1,0.0623,0.88700,...,0,0,0,0,0,1,0,0,0,0


In [49]:
# Filling in any null values. (In case)

df_2 = df_2.fillna(0)
df_2.head()

,id,name,duration_ms,artists,danceability,energy,loudness,mode,speechiness,acousticness,...,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
0,4YnXlBbFnMJf6qiO2LQMlU,God knows...,279040,[涼宮ハルヒ (CV.平野 綾)],0.456,0.955,-2.225,1,0.1330,0.00152,...,0,0,0,0,0,0,0,0,0,1
1,6catF1lDhNTjjGa2GxRQNN,You'll Never Walk Alone - Mono; 2002 Remaster,160187,0,0.484,0.265,-11.101,1,0.0322,0.39400,...,0,0,0,0,0,0,0,0,0,0
3,4aSw1QJIMwYSoDEgzgdCJL,Ferry Cross the Mersey - Mono; 2002 Remaster,141987,0,0.405,0.365,-10.226,0,0.0289,0.25500,...,0,0,0,0,1,0,0,0,0,0
4,0ZMMtH875IR2TfkyC4PolD,Don't Let the Sun Catch You Crying (Main) - Mono,157093,0,0.477,0.352,-14.165,1,0.0300,0.40600,...,0,0,0,0,0,0,0,0,0,0
5,1hx7X9cMXHWJjknb9O6Ava,The September Of My Years - Live At The Sands ...,187333,0,0.319,0.201,-17.796,1,0.0623,0.88700,...,0,0,0,0,0,1,0,0,0,0


In [50]:
df_2.shape

(166251, 32)

### Weighting

In [51]:
numericals = {'danceability': 0.15, 'energy': 0.15, 'loudness': 0.1, 'mode': 0.05, 'speechiness': 0.05, 'acousticness': 0.1, 
    'instrumentalness': 0.1, 'liveness': 0.05, 'valence': 0.15, 'tempo': 0.1}
categoricals = {'key_': 0.2}
numericals_key = numericals.keys()
categoricals_key = categoricals.keys()

In [52]:
from sklearn.preprocessing import MinMaxScaler
# Normalize numerical features
numerical_features = ['loudness', 'tempo']
scaler = MinMaxScaler()
df_2[numerical_features] = scaler.fit_transform(df_2[numerical_features])

# Standardize Year
df_2['standardized_year'] = scaler.fit_transform(df_2[['year']])


In [53]:
for i in numericals_key:
    df_2[i] = df_2[i] * numericals[i]

In [54]:
for n in categoricals_key:
    for z in df_2.columns:
        if n in z:
            df_2[z] = df_2[z] * categoricals[n]

In [55]:
# # Applying weight to input language.
# df_2['lang_' + language_prority] = df_2['lang_' + language_prority] * 10
# df_2[['name','lang_' + language_prority]].head()

In [56]:
# # Reducing weight from non-input language.
# for i in df_2.columns:
#     if 'lang_' in i:
#         if i != ('lang_' + language_prority):
#             df_2[i] = df_2[i] / 10
# df_2.head()

In [57]:
# # Applying weight to input key.
# df_2['key_' + str(key_priority)] = df_2['key_' + str(key_priority)] * 10
# df_2[['name','key_' + str(key_priority)]].head()

In [58]:
# # Reducing weight from non-input key.
# for i in df_2.columns:
#     if 'key_' in i:
#         if i != ('key_' + str(key_priority)):
#             df_2[i] = df_2[i] / 10
# df_2['key_7'].head()

In [59]:
# # Applying weight to input genre(s).
# superset_genre_priority = list(set(superset_genre_priority))

# if len(superset_genre_priority) != 0:
#     for i in superset_genre_priority:
#         df_2[i] = df_2[i] * 10
# df_2[superset_genre_priority].head()

In [60]:
# # Reducing weight from non-input genre(s).
# for n in superset_genre_priority:
#     for i in df_2.columns:
#         if i in superset_mapping.keys():
#             if i != (n):
#                 df_2[i] = df_2[i] / 10
# df_2['pop'].head()

### Cosine Similarity & Output Extraction

In [61]:
df_2.columns

Index(['id', 'name', 'duration_ms', 'artists', 'danceability', 'energy',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'year', 'language', 'langauge',
       'song_id_name', 'genres_count', 'supersetted_genres', 'key_0', 'key_1',
       'key_2', 'key_3', 'key_4', 'key_5', 'key_6', 'key_7', 'key_8', 'key_9',
       'key_10', 'key_11', 'standardized_year'],
      dtype='object')

In [62]:
import numpy as np
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity

# Feature selection, removing non-useful rows.
def processing(df_2):
    # Create a feature matrix
    feature_matrix = df_2.drop(['id', 'name', 'artists', 'song_id_name', 'language', 'langauge', 'year', 'supersetted_genres'], axis=1)

    # Apply normalization to the entire feature matrix
    feature_matrix = scaler.fit_transform(feature_matrix)

    # This is the input song. We are isolating it from the dataframe first.
    input_song = feature_matrix[0]
    
    arr = []

    # Returns comparison value of first song in dataset to all others in an array.
    # DataFrame can be customized before Feature Selection to change which song the subject of comparison should be.
    for i in range(1, len(feature_matrix)):
        # Take the i-th song from the feature matrix
        compare_song = feature_matrix[i]
        # Perform cosine similarity.
        cosine_sim = cosine_similarity([input_song], [compare_song])
        cs_value = cosine_sim[0][0]  # Take the single similarity value between the input song and the i-th song.
        arr.append(cs_value)

    return arr

arr = processing(df_2)
arr

[0.6717192142303055,
 0.5419992925209915,
 0.6830557194911523,
 0.592053033232159,
 0.391676531111923,
 0.7346759137005677,
 0.7389772974316793,
 0.6482187424705605,
 0.5022954819675923,
 0.716503811890526,
 0.6564035652363467,
 0.6486526416003785,
 0.731496141529834,
 0.6353339875881651,
 0.8043522980073619,
 0.5824729741162736,
 0.6374418955441052,
 0.6182388865713608,
 0.44278914073070846,
 0.42683426995653473,
 0.4242912434580696,
 0.7439128152488679,
 0.74026567048273,
 0.423263273657592,
 0.6856608904577264,
 0.6979341541284372,
 0.564191816830963,
 0.57877078413395,
 0.4583047357232752,
 0.5882984858331091,
 0.4761538873659806,
 0.5637148797027813,
 0.7268515946215676,
 0.6548790340895249,
 0.7205088477614114,
 0.5887046124803149,
 0.7204908202193783,
 0.6954053141387965,
 0.6952082511302413,
 0.7509630554855224,
 0.5583559542678626,
 0.6755084370482058,
 0.7468330226710648,
 0.5614204329649202,
 0.7220267888521688,
 0.6374779817890095,
 0.7067640035434484,
 0.574108862060297,
 

In [63]:
max(arr[1:])

0.9961737213246591

In [64]:
def compile_suggestions(arr):
    suggestions = []
    
    # Compiling all song ids that are above the pre-determined threshold for 'like the first song'.
    for i in range(len(arr)):
        if arr[i] > 0.93:
            suggestions.append(i)
    
    print(suggestions)
    return suggestions
suggestions = compile_suggestions(arr)

[307, 308, 312, 326, 355, 378, 380, 391, 396, 416, 424, 437, 467, 585, 588, 634, 727, 728, 805, 949, 967, 1103, 1104, 1283, 1395, 1397, 1401, 1437, 1462, 1493, 1539, 1578, 1624, 1701, 1727, 1728, 1833, 1857, 1884, 2093, 2140, 2250, 2266, 2286, 2296, 2356, 2358, 2366, 2427, 2428, 2465, 2489, 2502, 2522, 2523, 2532, 2550, 2569, 2605, 2606, 2638, 2725, 2734, 2741, 2771, 2796, 2846, 2907, 2932, 3001, 3036, 3161, 3163, 3220, 3222, 3241, 3319, 3326, 3333, 3363, 3412, 3448, 3485, 3671, 3698, 3707, 3718, 3733, 3745, 3786, 3803, 3851, 3904, 3917, 3932, 3934, 3975, 4061, 4108, 4115, 4116, 4157, 4259, 4416, 4419, 4479, 4491, 4568, 4619, 4701, 4772, 4782, 4821, 4853, 4926, 4937, 4981, 5010, 5122, 5238, 5378, 5392, 5397, 5427, 5463, 5513, 5531, 5578, 5587, 5595, 5653, 5664, 5674, 5782, 5788, 5793, 5857, 5868, 5912, 5941, 5966, 5994, 5995, 6017, 6032, 6288, 6295, 6298, 6317, 6322, 6341, 6378, 6393, 6475, 6562, 6657, 6667, 6679, 6696, 6761, 6777, 6792, 6844, 6913, 6955, 6982, 6992, 7013, 7024, 7036, 

In [65]:
len(suggestions)

3182

In [66]:
# Extracting out the end result information.
df_main = df_2[df_2.index.isin(suggestions)]

df_main.head()

,id,name,duration_ms,artists,danceability,energy,loudness,mode,speechiness,acousticness,...,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,standardized_year
307,4gT2koXscd2LtgtF2zkBg5,Whole Lot of Shakin' Goin' On - 2008 Remaster,186373,0,0.07320,0.11010,0.075666,0.00,0.002075,0.0510,...,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.380952
308,745i7maunSAJUAWQc94Asf,Looking for My Life - Mono; 2008 Remaster,166827,0,0.07710,0.05325,0.077215,0.05,0.001705,0.0755,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.380952
312,6BrUKXMND0QD2HXN1Wp8uP,Today I'm in Love - Stereo,120573,0,0.07095,0.11535,0.075652,0.05,0.002450,0.0400,...,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.380952
326,558D3GX2TjNixp21ZCjTvN,THE YEAR 3000,114888,0,0.09240,0.12630,0.072665,0.00,0.005150,0.0357,...,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000
355,0yieinYQlN2K0KL9T8xMXv,Pimpin n tha Year 3000,200600,0,0.13395,0.10980,0.076595,0.00,0.007150,0.0423,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.666667


In [67]:
len(df_main) # There is one less for the last 3 features as the input song did not go through the extracted year's steps.
# df_final['standardized_year'].describe()

2827

In [68]:
df_main = df_main.iloc[1:]

df_main = df_main.drop('artists', axis=1)

id_artist_mapping = df.set_index('id')['artists'].to_dict()

# Add 'artist' column to df1 using the mapping
df_main['artists'] = df_main['id'].map(id_artist_mapping)
df_main.head()

,id,name,duration_ms,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,...,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11,standardized_year,artists
308,745i7maunSAJUAWQc94Asf,Looking for My Life - Mono; 2008 Remaster,166827,0.07710,0.05325,0.077215,0.05,0.001705,0.0755,0.000000e+00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.380952,['Gerry & The Pacemakers']
312,6BrUKXMND0QD2HXN1Wp8uP,Today I'm in Love - Stereo,120573,0.07095,0.11535,0.075652,0.05,0.002450,0.0400,0.000000e+00,...,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.380952,['Gerry & The Pacemakers']
326,558D3GX2TjNixp21ZCjTvN,THE YEAR 3000,114888,0.09240,0.12630,0.072665,0.00,0.005150,0.0357,4.120000e-02,...,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,"['The Fifth Reality', 'Benjamin']"
355,0yieinYQlN2K0KL9T8xMXv,Pimpin n tha Year 3000,200600,0.13395,0.10980,0.076595,0.00,0.007150,0.0423,3.750000e-07,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,"['Suga Free', 'J Steez']"
378,6FLwmdmW77N1Pxb1aWsZmO,Only Time,218547,0.06270,0.03735,0.073593,0.05,0.001505,0.0841,6.610000e-02,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,['Enya']


### Hard Category Filtering

In [69]:
# Keeping only the songs that match the input song's language.
df_filter = pd.DataFrame()

# Filter rows where the genre is present in the 'supersetted_genres' column
language_filter = df_main[df_main['language'].apply(lambda x: language_prority == x)]
# Concatenate filtered DataFrame with df_filter
df_filter = pd.concat([df_filter, language_filter])

In [70]:
# Keeping only the songs that match the input song's genre(s)
# Iterate over each genre in the priority list
for genre in genre_priority:
    # Filter rows where the genre is present in the 'supersetted_genres' column
    genre_filter = df_filter[df_filter['supersetted_genres'].apply(lambda x: genre in x)]
    # Concatenate filtered DataFrame with df_filter
    df_filter = pd.concat([df_filter, genre_filter])

In [71]:
print(df_filter.shape)
df_filter.drop_duplicates(subset=["id"], keep='first', inplace=True)
print(df_filter.shape)

(6, 33)
(6, 33)


In [72]:
df_filter.iloc[:,1:19].head()

,name,duration_ms,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,year,language,langauge,song_id_name,genres_count,supersetted_genres
27560,トラウマティック・ガール,204493,0.0486,0.14400,0.090831,0.05,0.006600,0.02280,0.000456,0.00645,0.04965,0.064758,2016,ja,ja,4TwLJRCmdSjLFk3C7fDsR6トラウマティック・ガール,1,[]
49509,Prototipo,228676,0.0756,0.08535,0.088091,0.00,0.001890,0.08400,0.000000,0.00740,0.05130,0.075198,2014,ja,ja,2eDnDmBCMRWiADTOWgVtHRPrototipo,2,[]
69154,オレンジ,350200,0.0534,0.07335,0.087751,0.05,0.001615,0.01770,0.000000,0.00580,0.04785,0.077946,2011,ja,ja,7p30D2KjK2XW7YUvI6nH3oオレンジ,5,[anime]
70553,アイノカタチ,262693,0.0585,0.09810,0.085520,0.05,0.001675,0.00241,0.000000,0.00980,0.04560,0.081801,2016,ja,ja,76flsS3UnNr7UQIz6JmVT3アイノカタチ,1,[]
122010,手紙〜拝啓十五の君へ〜合唱バージョン - Gassho Version,314013,0.0912,0.06225,0.083059,0.05,0.001430,0.08370,0.000000,0.01185,0.04845,0.057296,2010,ja,ja,5wZ5BRly31ta2MVMpkUDkF手紙〜拝啓十五の君へ〜合唱バージョン - Gas...,2,[]


### Recommendations Output

In [73]:
# Generating recc_num of the matched songs at random.
def recommend(df_filter, recc_num):
    recs = []
    if len(df_filter) == 0:
        print("Sorry! There are no songs similar enough to " + input_song + "!")
    elif len(df_filter) < recc_num:
        recc_num = len(df_filter)
        print("Uh oh! We only found " + str(recc_num) + " recommendations!")
    print('Here are your recommendations!')
    for c in range(recc_num):
        recc = df_filter.sample().to_dict()
        name = list(recc['name'].values())[0]
        artists = (list(recc['artists'].values())[0])[1:len(list(recc['artists'].values())[0])-1]
        year = str(list(recc['year'].values())[0])
        print(str(c + 1) + '. ' + name + ' by ' + artists + ' published in ' + year)
        df_filter = df_filter.drop(list(recc['id'])[0])
        recs.append((name, year))
    return recs
# May want to try a filter for repreventing duplicate songs in the list.
recs_for_user = recommend(df_filter, recc_num) # End Product

Uh oh! We only found 6 recommendations!
Here are your recommendations!
1. 手紙〜拝啓十五の君へ〜合唱バージョン - Gassho Version by 'Angela Aki' published in 2010
2. オレンジ by '7!!' published in 2011
3. めぐる季節(「魔女の宅急便」) by 'Kanako Oba' published in 2014
4. Prototipo by 'Nanpa Básico' published in 2014
5. アイノカタチ by 'MISIA', 'HIDE (GReeeeN)' published in 2016
6. トラウマティック・ガール by 'Scoobie Do' published in 2016


In [74]:
user_id = sp.current_user()["id"]
song_uris = []
for song in recs_for_user:
    result = sp.search(q=f"track:{song[0]} year:{format(float(song[1]), '.0f')}", type="track")
    print(result)
    try:
        uri = result["tracks"]["items"][0]["uri"]
        song_uris.append(uri)
    except IndexError:
        print(f"{song[0]} doesn't exist in Spotify. Skipped.")

playlist = sp.user_playlist_create(user=user_id, name=input_playlist, public=False)
sp.playlist_add_items(playlist_id=playlist["id"], items=song_uris)

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3A%E6%89%8B%E7%B4%99%E3%80%9C%E6%8B%9D%E5%95%93%E5%8D%81%E4%BA%94%E3%81%AE%E5%90%9B%E3%81%B8%E3%80%9C%E5%90%88%E5%94%B1%E3%83%90%E3%83%BC%E3%82%B8%E3%83%A7%E3%83%B3+-+Gassho+Version+year%3A2010&type=track&offset=0&limit=10', 'items': [], 'limit': 10, 'next': None, 'offset': 0, 'previous': None, 'total': 0}}
手紙〜拝啓十五の君へ〜合唱バージョン - Gassho Version doesn't exist in Spotify. Skipped.


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3A%E3%82%AA%E3%83%AC%E3%83%B3%E3%82%B8+year%3A2011&type=track&offset=0&limit=10', 'items': [{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3Zl0EsuYV23OgNw6WqGelN'}, 'href': 'https://api.spotify.com/v1/artists/3Zl0EsuYV23OgNw6WqGelN', 'id': '3Zl0EsuYV23OgNw6WqGelN', 'name': 'Momoiro Clover Z', 'type': 'artist', 'uri': 'spotify:artist:3Zl0EsuYV23OgNw6WqGelN'}], 'available_markets': ['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DK', 'DO', 'DE', 'EC', 'EE', 'SV', 'FI', 'FR', 'GR', 'GT', 'HN', 'HK', 'HU', 'IS', 'IE', 'IT', 'LV', 'LT', 'LU', 'MY', 'MT', 'MX', 'NL', 'NZ', 'NI', 'NO', 'PA', 'PY', 'PE', 'PH', 'PL', 'PT', 'SG', 'SK', 'ES', 'SE', 'CH', 'TW', 'TR', 'UY', 'US', 'GB', 'AD', 'LI', 'MC', 'ID', 'JP', 'TH', 'VN', 'RO', 'IL', 'ZA', 'SA', 'AE', 'BH', 'QA', 'OM', 'KW', 'EG', 'MA', 'DZ', 'TN', 'LB', 'JO', 'PS', 'IN', 'BY', 'KZ', '

{'snapshot_id': 'MiwzM2FkNTUzMWE0MmU4ZDdjNmYyYTQzMGNjZWM2YTA5NWUyNDdkOTU5'}